# 並行模型鏈 (Parallel Model Chains) - Gemini 版本

使用 Google Gemini 同時調用多個模型進行不同角度的分析。

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")
print("✅ Gemini 模型已建立")

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是專業的客服代表。"),
    ("human", "請回覆：\n\n{email_content}")
])

def format_email(reply):
    return f"親愛的客戶，\n\n{reply}\n\n此致\n客服團隊"

quality_check_chain = ChatPromptTemplate.from_messages([
    ("system", "評估郵件專業度（1-10分）。"),
    ("human", "{email}")
]) | model | StrOutputParser()

tone_analysis_chain = ChatPromptTemplate.from_messages([
    ("system", "分析郵件語氣。"),
    ("human", "{email}")
]) | model | StrOutputParser()

parallel_chain = RunnableParallel(
    formatted=RunnableLambda(format_email),
    quality=RunnableLambda(lambda x: {"email": format_email(x)}) | quality_check_chain,
    tone=RunnableLambda(lambda x: {"email": format_email(x)}) | tone_analysis_chain
)

chain = prompt_template | model | StrOutputParser() | parallel_chain
print("✅ 並行鏈已建立")

In [ ]:
result = chain.invoke({"email_content": "產品包裝損壞，可以退換貨嗎？"})
print(f"郵件：\n{result['formatted']}\n\n品質：{result['quality']}\n\n語氣：{result['tone']}")